# Лабораторная работа №2 Машинное обучение на больших данных
## Вариант: Задача регрессии - RandomForest, бинарной классификации - LogisticRegression


## 1. Инициализация PySpark фреймворка

In [1]:
import numpy as np
import pandas as pd
import os

Импорт библиотек Spark SQL и Spark ML

In [2]:
from pyspark.sql.types import *
from pyspark.sql.functions import *
from pyspark.sql import SparkSession

from pyspark.ml import Pipeline
from pyspark.ml.feature import VectorAssembler, StringIndexer, VectorIndexer, MinMaxScaler, OneHotEncoder
from pyspark.ml.classification import LogisticRegression
from pyspark.ml.tuning import ParamGridBuilder, CrossValidator
from pyspark.ml.evaluation import BinaryClassificationEvaluator, RegressionEvaluator
from pyspark.ml.regression import RandomForestRegressor

spark = SparkSession.builder.master("local[*]").getOrCreate()

Загрузка исходных данных

In [3]:
dataframe = spark.read.csv('new_csv/itineraries2.csv', inferSchema=True, header=True, sep=';')
dataframe.limit(5).toPandas()

,startingAirport,destinationAirport,isNonStop,isBasicEconomy,isRefundable,baseFare,totalFare,seatsRemaining,totalTravelDistance,travelDuration
0,ATL,BOS,True,False,False,217.67,248.6,9,947,149
1,ATL,BOS,True,False,False,217.67,248.6,4,947,150
2,ATL,BOS,True,False,False,217.67,248.6,9,947,150
3,ATL,BOS,True,False,False,217.67,248.6,8,947,152
4,ATL,BOS,True,False,False,217.67,248.6,9,947,154


# Задача регрессии
## Прогнозирование длительности полета

### Подготовка данных

In [4]:
data = dataframe.select(
    "startingAirport", 
    "destinationAirport", 
    "isNonStop", 
    "isBasicEconomy", 
    "isRefundable", 
    "baseFare",
    "totalFare", 
    "seatsRemaining", 
    "totalTravelDistance",
    ((col("travelDuration")).alias("label")))
data.show(10)

+---------------+------------------+---------+--------------+------------+--------+---------+--------------+-------------------+-----+
|startingAirport|destinationAirport|isNonStop|isBasicEconomy|isRefundable|baseFare|totalFare|seatsRemaining|totalTravelDistance|label|
+---------------+------------------+---------+--------------+------------+--------+---------+--------------+-------------------+-----+
|            ATL|               BOS|     true|         false|       false|  217.67|    248.6|             9|                947|  149|
|            ATL|               BOS|     true|         false|       false|  217.67|    248.6|             4|                947|  150|
|            ATL|               BOS|     true|         false|       false|  217.67|    248.6|             9|                947|  150|
|            ATL|               BOS|     true|         false|       false|  217.67|    248.6|             8|                947|  152|
|            ATL|               BOS|     true|         

## Разделим данные

Используем 70% данных для обучения, а 30% оставим для тестирования.

In [5]:
splits = data.randomSplit([0.7, 0.3])
train = splits[0]
test = splits[1].withColumnRenamed("label", "trueLabel")
train_rows = train.count()
test_rows = test.count()
print("Training Rows:", train_rows, " Testing Rows:", test_rows)

Training Rows: 652911  Testing Rows: 278634


Задача регрессии (Случайный лес)

In [6]:
## Создание столбца признаков для задачи регрессии
# StringIndexer - преобразование текстовых значений в категориальных столбцах в числовые индексы
# OneHotEncoder - OHE-кодирование числовых индексов от StringIndexer
# VectorAssembler - объединение всех категориальных признаков в один вектор
# VectorIndexer - автоматическое определение категориальных признаков вектора
# MinMaxScaler - масштабирование числовых признаков в диапазон от 0 до 1
strIdx = StringIndexer(inputCols=["startingAirport", "destinationAirport"], outputCols=["startingAirport_idx", "destinationAirport_idx"])
oneHotEnc = OneHotEncoder(inputCols=["startingAirport_idx", "destinationAirport_idx"], outputCols=["startingAirport_enc", "destinationAirport_enc"])
catVect = VectorAssembler(inputCols=["startingAirport_enc", "destinationAirport_enc", "isNonStop", "isBasicEconomy", "isRefundable", "seatsRemaining"], outputCol="reg_catFeatures")
catIdx = VectorIndexer(inputCol=catVect.getOutputCol(), outputCol="reg_idxCatFeatures")
numVect = VectorAssembler(inputCols=["baseFare", "totalFare", "totalTravelDistance"], outputCol="reg_numFeatures")
minMax = MinMaxScaler(inputCol=numVect.getOutputCol(), outputCol='reg_normFeatures')
featVect = VectorAssembler(inputCols=["reg_idxCatFeatures", "reg_numFeatures", "reg_normFeatures"], outputCol="reg_features")

# Создание модели RandomForestRegressor
rf = RandomForestRegressor(labelCol="label", featuresCol="reg_features", numTrees=10)

# Создание и выполнение пайплайна для задачи регрессии
reg_pipeline = Pipeline(stages=[strIdx, oneHotEnc, catVect, catIdx, numVect, minMax, featVect, rf])
reg_model = reg_pipeline.fit(train)
reg_prediction = reg_model.transform(test)

# Вывод результатов
reg_prediction.select("reg_features", "prediction", "trueLabel").show(10, truncate=False)

+----------------------------------------------------------------------------------------------------------------------------+------------------+---------+
|reg_features                                                                                                                |prediction        |trueLabel|
+----------------------------------------------------------------------------------------------------------------------------+------------------+---------+
|(40,[8,18,34,35,36,37,38,39],[1.0,1.0,161.86,196.1,956.0,0.20835374149659863,0.21730019062527614,0.2176179306750065])       |392.10998182062906|252      |
|(40,[8,18,33,34,35,36,37,38,39],[1.0,1.0,1.0,161.86,196.1,956.0,0.20835374149659863,0.21730019062527614,0.2176179306750065])|392.10998182062906|252      |
|(40,[8,18,33,34,35,36,37,38,39],[1.0,1.0,2.0,161.86,196.1,956.0,0.20835374149659863,0.21730019062527614,0.2176179306750065])|392.10998182062906|323      |
|(40,[8,18,33,34,35,36,37,38,39],[1.0,1.0,3.0,161.86,196.1,956.0

## Оценка качества модели

In [7]:
# RMSE - среднеквадратичное отклонение между фактическими и предсказанными значениями
reg_evaluator = RegressionEvaluator(labelCol="trueLabel", predictionCol="prediction", metricName="rmse")
rmse = reg_evaluator.evaluate(reg_prediction)
print("RMSE = ", rmse)

RMSE =  109.47270679506029


In [8]:
# R² - доля общей дисперсии
reg_evaluator = RegressionEvaluator(labelCol="trueLabel", predictionCol="prediction", metricName="r2")
r2 = reg_evaluator.evaluate(reg_prediction)
print("R² = ", r2)

R² =  0.7145097796726563


In [9]:
# MAE - абсолютное отклонение между фактическими и предсказанными значениями
mae_evaluator = RegressionEvaluator(labelCol="trueLabel", predictionCol="prediction", metricName="mae")
mae = mae_evaluator.evaluate(reg_prediction)
print(f"Mean Absolute Error (MAE): {mae}")

Mean Absolute Error (MAE): 78.03904788279135


In [10]:
# Стандартное отклонение
data.select(stddev('label')).show()

+------------------+
|     stddev(label)|
+------------------+
|204.51993614793133|
+------------------+



CrossValidator разделяет обучающий набор данных на заданное количество складок (фолдов), обучает модель на части данных и оценивает ее на оставшихся данных. Этот процесс повторяется для каждой комбинации гиперпараметров, и выбирается модель с наилучшей средней производительностью на всех фолдах.

In [11]:
## Настройка гиперпараметров модели
# maxDepth - максимальная глубина дерева, выбор наилучшего 2,5,10
# numTrees - количество деревьев в случайном лесу, выбор наилучшего из 5,10,20
param_grid = (ParamGridBuilder()
              .addGrid(rf.maxDepth, [2, 5, 10])
              .addGrid(rf.numTrees, [5, 10, 20])
              .build())

## Создание объекта для кросс-валидации
# estimator - оцениваемая модель (наш пайплайн c моделью и преобразованием данных)
# estimatorParamMaps - сетка параметров для оцениваемой модели
# evaluator - оценка качества модели на каждом этапе кросс-валидации (по умолчанию RMSE) 
# numFolds - количество складываний для кросс-валидации, т.е. на сколько частей будет разделен набор данных (в нашем случае разделение на 3 части и обучение 3 раза)
cv = CrossValidator(estimator=reg_pipeline,
                            estimatorParamMaps=param_grid,
                            evaluator=RegressionEvaluator(),
                            numFolds=3) 

# Обучение модели с использованием кросс-валидации
cv_model = cv.fit(train)

# Получение наилучшей модели после обучения с кросс-валидацией
best_cv_model = cv_model.bestModel

# Оценка производительности на тестовом наборе
cv_prediction = best_cv_model.transform(test)

In [12]:
reg_evaluator = RegressionEvaluator(labelCol="trueLabel", predictionCol="prediction", metricName="rmse")
rmse = reg_evaluator.evaluate(cv_prediction)
print("RMSE = ", rmse)

RMSE =  100.6302591302342


In [13]:
# Расчет метрик для задачи регрессии
reg_evaluator = RegressionEvaluator(labelCol="trueLabel", predictionCol="prediction", metricName="r2")
r2 = reg_evaluator.evaluate(cv_prediction)
print("R² = ", r2)

R² =  0.7587670071785422


In [14]:
mae_evaluator = RegressionEvaluator(labelCol="trueLabel", predictionCol="prediction", metricName="mae")
mae = mae_evaluator.evaluate(cv_prediction)
print(f"Mean Absolute Error (MAE): {mae}")

Mean Absolute Error (MAE): 69.3906688369729


# Задача бинарной классификации

## Подготовка данных


In [15]:
# 0 - для рейса с длительностью полета менее 400 минут,
# 1 - для рейса с длительностью полета более 400 минут
data = dataframe.select(
    "startingAirport", 
    "destinationAirport", 
    "isNonStop", 
    "isBasicEconomy", 
    "isRefundable", 
    "baseFare",
    "totalFare", 
    "seatsRemaining", 
    "totalTravelDistance",
    ((col("travelDuration") > 375).cast("Int").alias("label")))
data.show(10)

+---------------+------------------+---------+--------------+------------+--------+---------+--------------+-------------------+-----+
|startingAirport|destinationAirport|isNonStop|isBasicEconomy|isRefundable|baseFare|totalFare|seatsRemaining|totalTravelDistance|label|
+---------------+------------------+---------+--------------+------------+--------+---------+--------------+-------------------+-----+
|            ATL|               BOS|     true|         false|       false|  217.67|    248.6|             9|                947|    0|
|            ATL|               BOS|     true|         false|       false|  217.67|    248.6|             4|                947|    0|
|            ATL|               BOS|     true|         false|       false|  217.67|    248.6|             9|                947|    0|
|            ATL|               BOS|     true|         false|       false|  217.67|    248.6|             8|                947|    0|
|            ATL|               BOS|     true|         

## Разделим данные

Используем 70% данных для обучения, а 30% оставим для тестирования..

In [16]:
splits = data.randomSplit([0.7, 0.3])
train = splits[0]
test = splits[1].withColumnRenamed("label", "trueLabel")
train_rows = train.count()
test_rows = test.count()
print("Training Rows:", train_rows, " Testing Rows:", test_rows)

Training Rows: 652077  Testing Rows: 279468


## Вычисление отношения между классами

In [17]:
positive_count = train.filter(col("label") == 1).count()
negative_count = train.filter(col("label") == 0).count()
balance_ratio = positive_count / negative_count
print("Positive to Negative Class Ratio:", balance_ratio)

Positive to Negative Class Ratio: 1.0926531516063709


Если balance_ratio близко к 1, это свидетельствует о балансе. Если значительно больше или меньше 1, это может указывать на дисбаланс.

In [18]:
# Создание столбца признаков для задачи регрессии
strIdx = StringIndexer(inputCols=["startingAirport", "destinationAirport"], outputCols=["startingAirport_idx", "destinationAirport_idx"])
oneHotEnc = OneHotEncoder(inputCols=["startingAirport_idx", "destinationAirport_idx"], outputCols=["startingAirport_enc", "destinationAirport_enc"])
catVect = VectorAssembler(inputCols=["startingAirport_enc", "destinationAirport_enc", "isNonStop", "isBasicEconomy", "isRefundable", "seatsRemaining"], outputCol="reg_catFeatures")
catIdx = VectorIndexer(inputCol=catVect.getOutputCol(), outputCol="reg_idxCatFeatures")
numVect = VectorAssembler(inputCols=["baseFare", "totalFare", "totalTravelDistance"], outputCol="reg_numFeatures")
minMax = MinMaxScaler(inputCol=numVect.getOutputCol(), outputCol='reg_normFeatures')
featVect = VectorAssembler(inputCols=["reg_idxCatFeatures", "reg_numFeatures", "reg_normFeatures"], outputCol="features")
lr = LogisticRegression(labelCol="label",featuresCol="features",maxIter=10,regParam=0.3)
pipeline = Pipeline(stages=[strIdx, oneHotEnc, catVect, catIdx, numVect, minMax, featVect, lr])

In [19]:
pipelineModel = pipeline.fit(train)

In [20]:
prediction = pipelineModel.transform(test)
predicted = prediction.select("features", "prediction", "trueLabel")
predicted.show(100, truncate=False)

+------------------------------------------------------------------------------------------------------------------------------+----------+---------+
|features                                                                                                                      |prediction|trueLabel|
+------------------------------------------------------------------------------------------------------------------------------+----------+---------+
|(40,[8,18,34,35,36,37,38,39],[1.0,1.0,161.86,196.1,956.0,0.20835374149659863,0.21730019062527614,0.2176179306750065])         |0.0       |0        |
|(40,[8,18,34,35,36,37,38,39],[1.0,1.0,161.86,196.1,956.0,0.20835374149659863,0.21730019062527614,0.2176179306750065])         |0.0       |0        |
|(40,[8,18,33,34,35,36,37,38,39],[1.0,1.0,1.0,161.86,196.1,956.0,0.20835374149659863,0.21730019062527614,0.2176179306750065])  |0.0       |0        |
|(40,[8,18,33,34,35,36,37,38,39],[1.0,1.0,1.0,161.86,196.1,956.0,0.20835374149659863,0.2173001906252

In [21]:
tp = float(predicted.filter("prediction == 1.0 AND truelabel == 1").count())
fp = float(predicted.filter("prediction == 1.0 AND truelabel == 0").count())
tn = float(predicted.filter("prediction == 0.0 AND truelabel == 0").count())
fn = float(predicted.filter("prediction == 0.0 AND truelabel == 1").count())
pr = tp / (tp + fp)
re = tp / (tp + fn)
metrics = spark.createDataFrame([
 ("TP", tp),
 ("FP", fp),
 ("TN", tn),
 ("FN", fn),
 ("Precision", pr),
 ("Recall", re),
 ("F1", 2*pr*re/(re+pr))],["metric", "value"])
metrics.show()

+---------+------------------+
|   metric|             value|
+---------+------------------+
|       TP|          123775.0|
|       FP|           30391.0|
|       TN|          103205.0|
|       FN|           22097.0|
|Precision|0.8028683367279426|
|   Recall| 0.848517878688165|
|       F1|0.8250621587932194|
+---------+------------------+



In [22]:
# Метрика Area Under the ROC Curve
evaluator = BinaryClassificationEvaluator(labelCol="trueLabel", rawPredictionCol="rawPrediction", metricName="areaUnderROC")
aur = evaluator.evaluate(prediction)
print ("AUR = ", aur)

AUR =  0.916899057270512


## Подбор гиперпараметров

In [23]:
## Настройка гиперпараметров
# regParam - коэфф регуляции
# maxIter - максимальное кол-во итераций при обучении
# threshold порог классификации
paramGrid = (ParamGridBuilder()
             .addGrid(lr.regParam, [0.3, 0.2])
             .addGrid(lr.maxIter, [15, 10])
             .addGrid(lr.threshold,[0.45, 0.35])
             .build())

cv = CrossValidator(estimator=pipeline, 
                    evaluator=BinaryClassificationEvaluator(), 
                    estimatorParamMaps=paramGrid, 
                    numFolds=2)

model = cv.fit(train)

In [24]:
newPrediction = model.transform(test)
newPredicted = prediction.select("features", "prediction", "trueLabel")
newPredicted.show()

+--------------------+----------+---------+
|            features|prediction|trueLabel|
+--------------------+----------+---------+
|(40,[8,18,34,35,3...|       0.0|        0|
|(40,[8,18,34,35,3...|       0.0|        0|
|(40,[8,18,33,34,3...|       0.0|        0|
|(40,[8,18,33,34,3...|       0.0|        0|
|(40,[8,18,33,34,3...|       0.0|        0|
|(40,[8,18,33,34,3...|       0.0|        1|
|(40,[8,18,34,35,3...|       0.0|        0|
|(40,[8,18,34,35,3...|       0.0|        0|
|(40,[8,18,33,34,3...|       0.0|        0|
|(40,[8,18,33,34,3...|       0.0|        0|
|(40,[8,18,33,34,3...|       0.0|        0|
|(40,[8,18,33,34,3...|       0.0|        0|
|(40,[8,18,33,34,3...|       0.0|        0|
|(40,[8,18,33,34,3...|       0.0|        0|
|(40,[8,18,33,34,3...|       0.0|        1|
|(40,[8,18,33,34,3...|       0.0|        1|
|(40,[8,18,33,34,3...|       0.0|        0|
|(40,[8,18,33,34,3...|       0.0|        0|
|(40,[8,18,33,34,3...|       0.0|        1|
|(40,[8,18,33,34,3...|       0.0

In [25]:
# Recalculate confusion matrix
tp2 = float(newPrediction.filter("prediction == 1.0 AND truelabel == 1").count())
fp2 = float(newPrediction.filter("prediction == 1.0 AND truelabel == 0").count())
tn2 = float(newPrediction.filter("prediction == 0.0 AND truelabel == 0").count())
fn2 = float(newPrediction.filter("prediction == 0.0 AND truelabel == 1").count())
pr2 = tp2 / (tp2 + fp2)
re2 = tp2 / (tp2 + fn2)
metrics2 = spark.createDataFrame([
 ("TP", tp2),
 ("FP", fp2),
 ("TN", tn2),
 ("FN", fn2),
 ("Precision", pr2),
 ("Recall", re2),
 ("F1", 2*pr2*re2/(re2+pr2))],["metric", "value"])
metrics2.show()

+---------+------------------+
|   metric|             value|
+---------+------------------+
|       TP|          133773.0|
|       FP|           40758.0|
|       TN|           92838.0|
|       FN|           12099.0|
|Precision|0.7664712859033639|
|   Recall|0.9170574202040145|
|       F1|0.8350296345539836|
+---------+------------------+



In [26]:
# Recalculate the Area Under ROC
evaluator2 = BinaryClassificationEvaluator(labelCol="trueLabel", rawPredictionCol="prediction", metricName="areaUnderROC")
aur2 = evaluator.evaluate(prediction)
print("AUR2 = ", aur2)

AUR2 =  0.9168969351605538
